In [ ]:
%pip install neo4j pandas

The data has been imported into Neo4j, a Graph Database.  Graph Databases allow you to look at the data in terms of Nodes and Relationships rather than rows and columns.  

![Data Model](images/model.svg)

[Cypher](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_overview/), Neo4j's query language allows you to find patterns in the chain of events.   For example, a specific pattern:

```cypher
MATCH sequence = 
(:Carry)-[:NEXT_EVENT]->(:Pass)-[:NEXT_EVENT]->(:BallReceipt)-[:NEXT_EVENT]->(:Carry)-[:NEXT_EVENT]->(:Pass)-[:NEXT_EVENT]->(:BallReceipt)
```

Or a non-specific pattern of events that start and end with a particular event type.

```cypher
MATCH sequence = (:BallReceipt)-[:NEXT_EVENT*1..3]->(:BallReceipt)
```

![Pass Sequence](images/pass-sequence.svg)

In the case of the StatsBomb data, there may be many variable event types between a Ball Receipt and a Pass, for example: Ball Receipt, Carry, Dribble, Pass or as simple as Ball Receipt, Pass.

The events are linked to a Match node, which is linked to a Stadium node.  The Stadium node holds the pitch dimensions and a ratio (width: `120/actual width`, length: `80/actual length`) which is used to calculate the actual distance.

_The data has been imported in a separate notebook._


## Connecting to Neo4j

The following code creates an instance of the [Neo4j Python Driver](https://neo4j.com/docs/python-manual/current/), through which we can interact with the Neo4j database.

In [37]:
# Connect to Neo4j
from neo4j import GraphDatabase, Result

driver = GraphDatabase.driver(
    "neo4j://localhost:7687", 
    auth=("neo4j", "neoneoneo")
)

driver.verify_connectivity()

## Pitch Dimensions

The following code takes the pitch dimension information provided and assigns a `dimensions` property to the stadium node and calculates a ratio.  The two properties are `point` properties, which contain an `x` and `y`.  This will be used to adjust the locations in the Statsbomb data.

In [ ]:
# Load Stadium Dimensions
import pandas as pd

pitch_df = pd.read_csv('import/pl-pitch-dimensions.csv')
        
driver.execute_query("""
    UNWIND $rows AS row
    MATCH (s:Stadium)
    WHERE trim(s.name) = trim(row.`Stadium `)
    SET s.dimensions = point({ x: row.`Pitch Length (m)`, y: row.`Pitch Width (m)`}),
    s.ratio = point({ x: s.dimensions.x / 120, y: s.dimensions.y / 80})
    RETURN count(*),  collect(s.name)
""", rows = [dict(row) for _, row in pitch_df.iterrows()])


## Event Types

The following list contains all values for an event `type` in the Statsbomb dataset.

In [45]:
event_types_df = driver.execute_query(
    "MATCH (e:Event) RETURN e.type AS type, count(e) AS count ORDER BY type ASC",
    database_="neo4j",
    result_transformer_=Result.to_df
)
event_types_df

,type,count
0,50/50,1443
1,6 Seconds,2
2,Backpass Pick,2
3,Bad Behaviour,336
4,Ball Receipt*,350011
5,Ball Recovery,35810
6,Block,15153
7,Carry,286254
8,Clearance,16258
9,Dangerous Play,58


This list has been combined with event outcomes to create a set of node labels that can be used to quickly identify events - eg. unsuccessful passes or shots conceded by the goalkeeper.

In [44]:
event_labels_df = driver.execute_query(
    "MATCH (e:Event) UNWIND labels(e) AS label RETURN label, count(distinct e) AS count ORDER BY label ASC",
    database_="neo4j",
    result_transformer_=Result.to_df
)
event_labels_df

,label,count
0,BadBehaviour,336
1,BallReceipt,350011
2,BallRecovery,35810
3,Block,15153
4,Carry,286254
5,Clearance,16258
6,Dispossessed,8555
7,Dribble,10404
8,DribbledPast,6011
9,Duel,25991


### Events that end a possession

In [49]:
driver.execute_query(
    "MATCH (:Possession)-[:LAST_EVENT]->(e:Event) RETURN e.type AS type, count(*) AS count",
    database_="neo4j",
    result_transformer_=Result.to_df
)

,type,count
0,Half Start,613
1,Duel,3950
2,Goal Keeper,6075
3,Block,4506
4,Ball Receipt*,15748
5,Miscontrol,3455
6,Pass,4352
7,Pressure,2401
8,Clearance,4127
9,Ball Recovery,606


In [64]:
next_df = driver.execute_query(
    """
        MATCH (p:Possession) 
        WHERE exists { (p)<-[:POSSESSION]-(:BallReceipt)-[:NEXT_BALL_RECEIPT]->() }
        MATCH (p)-[:LAST_EVENT]->(e:Event), 
            (e)-[:TEAM]->(t),
            (p)-[:NEXT_POSSESSION]->()-[:FIRST_EVENT]->(n)-[:TEAM]->(nt)
        RETURN e.type AS type, e.outcome AS outcome, 
            n.type AS next, t = nt AS same_team,
            count(*) AS count
        ORDER BY type, outcome, next
    """,
    database_="neo4j",
    result_transformer_=Result.to_df
)

next_df.to_csv("exports/next_event.csv")

next_df

,type,outcome,next,same_team,count
0,50/50,Lost,50/50,False,44
1,50/50,Lost,Ball Recovery,False,1
2,50/50,Lost,Ball Recovery,True,3
3,50/50,Lost,Pass,True,19
4,50/50,Success To Opposition,Ball Recovery,False,4
5,50/50,Success To Opposition,Ball Recovery,True,1
6,50/50,Success To Opposition,Pass,False,19
7,50/50,Success To Team,Ball Recovery,True,28
8,50/50,Success To Team,Pass,False,2
9,50/50,Success To Team,Pass,True,29


## Example Patterns

The following code executes a Cypher statement to find (`MATCH`) Ball Receipts (nodes with a label of `BallReceipt`), then follows the `NEXT_EVENT` relationships from one to three degrees to another ball receipt.  The result is filtered with a `WHERE` clause to make sure all of the nodes in the sequence are performed by the same team as the original ball receipt.

* The `leadsToShot` column is determined by whether the possession leads to a shot (`(:Shot)`) or an own goal (`(:OwnGoalFor)`)
* The `leadsToGoal` column is true when a goal is scored (the outcome property of the shot is `Goal`) or an own goal is scored (`(:OwnGoalFor)`).

The `RETURN` clause of the Cypher statement calculates the adjusted distance and the duration of the event.

In [68]:
df = None

with driver.session() as session:
    df = session.run("""
        MATCH sequence = (br1:BallReceipt)-[:NEXT_EVENT*1..3]->(br2:BallReceipt),
            (br1)-[:TEAM]->(t),
            (br1)-[:PLAYER]->(p1),
            (br2)-[:PLAYER]->(p2),
            (br1)-[:MATCH]->(m)-[:AT_STADIUM]->(s),
            (br1)-[:POSSESSION]->(pos)-[:LAST_EVENT]->(last)
        WHERE all(n in nodes(sequence) where (n)-[:TEAM]->(t))
        WITH *, 
            point({ x: br1.location.x * s.ratio.x, y: br1.location.y * s.ratio.y }) AS firstAdjustedLocation,
            point({ x: br2.location.x * s.ratio.x, y: br2.location.y * s.ratio.y }) AS secondAdjustedLocation,
            [ (e)-[:NEXT_EVENT*0..3]->(last) | e.type ] AS lastEvents
        RETURN 
            pos.id AS possession,
            t.name AS team,
            br1.id AS firstReceiptId, 
            p1.id AS passer, 
            p2.id AS receiver, 
            br1.timestamp AS firstReceiptTimestamp,
            [ n in nodes(sequence) | n.type ] AS eventsInSequence, 
            br1.location.x as firstReceiptLocationX,  br1.location.y as firstReceiptLocationY, 
            br2.location.x AS secondReceiptLocationX, br2.location.y AS secondReceiptLocationY,
            point.distance(br1.location, br2.location) AS passDistance, 
            firstAdjustedLocation.x AS firstAdjustedLocationX, firstAdjustedLocation.y AS firstAdjustedLocationY, 
            secondAdjustedLocation.x AS secondAdjustedLocationX, secondAdjustedLocation.y AS secondAdjustedLocationY, 
            point.distance(firstAdjustedLocation, secondAdjustedLocation) AS adjustedPassDistance, 
            time.truncate('day') + duration.between(br1.timestamp, br2.timestamp) AS duration,
            CASE WHEN last:Goalkeeper THEN last.type + ' - '+ last.goalkeeper_type ELSE last.type END AS lastEventType,
            last.outcome AS lastEventOutcome,
            exists { (pos)<-[:POSSESSION]-(:Shot) } or exists { (pos)<-[:POSSESSION]-(:OwnGoalFor) } AS leadsToShot,
            exists { (pos)<-[:POSSESSION]-(:Shot {outcome: "Goal"}) } or exists { (pos)<-[:POSSESSION]-(:OwnGoalFor) } AS leadsToGoal
    """).to_df()
    
    # Save 
    df.to_csv("exports/ball-receipts.csv")

df

,possession,team,firstReceiptId,passer,receiver,firstReceiptTimestamp,eventsInSequence,firstReceiptLocationX,firstReceiptLocationY,secondReceiptLocationX,...,firstAdjustedLocationX,firstAdjustedLocationY,secondAdjustedLocationX,secondAdjustedLocationY,adjustedPassDistance,duration,lastEventType,lastEventOutcome,leadsToShot,leadsToGoal
0,3837579--165,Southampton,f5146bf7-1330-46f7-a288-66e3f3c33c3a,3328,3066,00:55:04.308000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",47.7,28.2,62.0,...,40.545000,23.970,52.700000,57.885,36.027368,00:00:03.808000000+00:00,Goal Keeper - Shot Faced,None,True,False
1,3837579--160,Southampton,e30aa087-6fa6-4cbc-9488-e81dd2c9c3de,4626,49670,00:53:13.427000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",70.9,32.9,92.0,...,60.265000,27.965,78.200000,5.780,28.527854,00:00:03.988000000+00:00,Pass,Incomplete,False,False
2,3837579--159,Southampton,64f6215c-5f7c-4449-9cb9-b8a2b46a23fd,22357,4626,00:52:11.901000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",33.1,35.4,52.1,...,28.135000,30.090,44.285000,47.600,23.820634,00:00:05.217000000+00:00,Ball Recovery,None,False,False
3,3837579--159,Southampton,dcce8e80-3f4f-4cc4-aa30-39dbb10278f2,22357,3619,00:52:00.236000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",20.2,23.4,5.7,...,17.170000,19.890,4.845000,28.900,15.267145,00:00:02.005000000+00:00,Ball Recovery,None,False,False
4,3837579--159,Southampton,09c3b4ca-2a75-40a9-9cc3-839f57dda997,4626,24478,00:52:17.118000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",52.1,56.0,107.7,...,44.285000,47.600,91.545000,26.350,51.817662,00:00:04.255000000+00:00,Ball Recovery,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251058,3837246--2,Crystal Palace,670b93d8-0210-4757-8830-96f1a55685ed,8247,9000,00:00:01.560000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",38.1,36.4,93.9,...,32.067500,30.940,79.032500,7.820,52.347355,00:00:04.585000000+00:00,Carry,None,False,False
251059,3837246--6,Liverpool,34025752-a1e1-4c51-8977-dee3a525122f,24938,3247,00:03:52.359000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",70.4,19.5,64.3,...,59.253333,16.575,54.119167,33.745,17.921177,00:00:02.520000000+00:00,Pass,Incomplete,False,False
251060,3837246--10,Crystal Palace,c999f3cd-4741-471d-8cc2-418f5ffec358,13851,4590,00:05:28.328000000+00:00,"[Ball Receipt*, Pass, Ball Receipt*]",37.8,70.9,42.5,...,31.815000,60.265,35.770833,64.685,5.931696,00:00:01.033000000+00:00,Ball Receipt*,Incomplete,False,False
251061,3837246--25,Liverpool,cee61525-4a75-4c4c-afd4-b198631d28a9,3655,27158,00:11:42.455000000+00:00,"[Ball Receipt*, Carry, Pass, Ball Receipt*]",43.5,5.2,62.2,...,36.612500,4.420,52.351667,3.995,15.744904,00:00:01.205000000+00:00,Pass,Incomplete,False,False


## Next Passes

Save a new `:NEXT_BALL_RECEIPT` relationship based on the data above.

In [33]:
with driver.session() as session:
    res = session.run("""
        MATCH sequence = (br1:BallReceipt)-[:NEXT_EVENT*1..3]->(br2:BallReceipt),
            (br1)-[:TEAM]->(t),
            (br1)-[:MATCH]->(m)-[:AT_STADIUM]->(s)
        WHERE all(n in nodes(sequence) where (n)-[:TEAM]->(t))
        
        CALL {
            WITH br1, br2, s
            WITH br1, br2, 
                point({ x: br1.location.x * s.ratio.x, y: br1.location.y * s.ratio.y }) AS firstAdjustedLocation,
                point({ x: br2.location.x * s.ratio.x, y: br2.location.y * s.ratio.y }) AS secondAdjustedLocation
            
            MERGE (br1)-[r:NEXT_BALL_RECEIPT]->(br2)
            SET 
                r.distance = point.distance(br1.location, br2.location),
                r.adjustedDistance = point.distance(firstAdjustedLocation, secondAdjustedLocation),
                r.duration = duration.between(br1.timestamp, br2.timestamp)
        } IN TRANSACTIONS OF 1000 ROWS
        
        RETURN count(*) AS count
    """)
    
    print(res.single().get('count'))

251063


## Average duration per Possession

Use the `:NEXT_BALL_RECEIPT` relationships to calculate an average pass duration and provide the final outcome of the possession.

In [76]:
import pandas as pd
from neo4j import Result

df = driver.execute_query("""
    MATCH (r1:BallReceipt)-[r:NEXT_BALL_RECEIPT]->(r2)-[:POSSESSION]->(p),
        (r1)-[:TEAM]->(t),
        (r1)-[:MATCH]->(m),
        (p)-[:LAST_EVENT]->(last),
        (p)-[:NEXT_POSSESSION]->(npos)-[:FIRST_EVENT]->(nextFirst)
    CALL {
        WITH last
        MATCH p = ()-[:NEXT_EVENT*3]->(last)
        RETURN reduce(output = [], n in nodes(p) | output + [
            n.type 
            + CASE WHEN n.outcome IS NOT NULL THEN ' ('+ n.outcome +')' ELSE '' END 
            + CASE WHEN n.out THEN ' > out' ELSE '' END 
        ]) AS lastEvents
    }
    WITH p, t, m, last, nextFirst, lastEvents,
        count(r) AS events, 
        avg(r.distance) AS avgDistance, 
        avg(r.adjustedDistance) AS avgAdjustedDistance, 
        sum(r.distance) AS totalDistance,
        sum(r.adjustedDistance) AS totalAdjustedDistance, 
        avg(r.duration) AS avgDuration,
        sum(r.duration) AS totalDuration
    RETURN 
        m.name AS name, 
        t.name AS team, 
        p.id AS possession, 
        events,
        time.truncate('day') + avgDuration AS avgDuration, 
        time.truncate('day') + totalDuration AS totalDuration, 
        avgDistance, 
        totalDistance,
        avgAdjustedDistance, 
        totalAdjustedDistance,
        round(avgAdjustedDistance / avgDuration.milliseconds * 1000, 2) AS tempo,
        last.type AS lastType, 
        last.outcome AS lastOutcome,
        lastEvents,
        exists { (p)<-[:POSSESSION]-(:Shot) } or exists { (p)<-[:POSSESSION]-(:OwnGoalFor) } AS leadsToShot,
        exists { (p)<-[:POSSESSION]-(:Shot {outcome: "Goal"}) } or exists { (p)<-[:POSSESSION]-(:OwnGoalFor) } AS leadsToGoal,
        CASE 
          WHEN (p)<-[:POSSESSION]-(:Shot) OR (nextFirst)-[:TEAM]->(t) THEN 'Positive'
          WHEN any(type in lastEvents WHERE type in ['Foul Won']) AND (last)-[:TEAM]->(t) THEN 'Positive'
          WHEN last.outcome = "Incomplete" THEN 'Negative'
          WHEN any(type in lastEvents WHERE type in ['Dispossessed', 'Miscontrol', 'Interception', 'Clearance', 'Error', 'Pressure']) THEN 'Negative'
          WHEN NOT (nextFirst)-[:TEAM]->(t) THEN 'Negative'
        END AS outcome
""", result_transformer_=Result.to_df)

df.to_csv('exports/possessions.csv')

df

,name,team,possession,events,avgDuration,totalDuration,avgDistance,totalDistance,avgAdjustedDistance,totalAdjustedDistance,tempo,lastType,lastOutcome,lastEvents,leadsToShot,leadsToGoal,outcome
0,2023-05-08 Nottingham Forest v Southampton,Nottingham Forest,3837579--32,2,00:00:02.791000000+00:00,00:00:05.582000000+00:00,46.994486,93.988972,39.945313,79.890626,14.31,Goal Keeper,Touched In,"[Pass, Ball Receipt*, Shot (Goal), Goal Keeper...",True,True,Positive
1,2023-05-08 Nottingham Forest v Southampton,Nottingham Forest,3837579--36,12,00:00:02.466583333+00:00,00:00:29.599000000+00:00,15.902714,190.832567,13.517307,162.207682,5.48,Goal Keeper,No Touch,"[Pass, Ball Receipt*, Shot (Goal), Goal Keeper...",True,True,Positive
2,2023-05-08 Nottingham Forest v Southampton,Southampton,3837579--40,2,00:00:03.311500000+00:00,00:00:06.623000000+00:00,31.038661,62.077322,26.382862,52.765724,7.97,Goal Keeper,No Touch,"[Ball Receipt*, Carry, Shot (Goal), Goal Keepe...",True,True,Positive
3,2023-05-08 Nottingham Forest v Southampton,Nottingham Forest,3837579--124,6,00:00:02.163500000+00:00,00:00:12.981000000+00:00,19.128571,114.771426,16.259285,97.555712,7.52,Goal Keeper,No Touch,"[Pass, Ball Receipt*, Shot (Goal), Goal Keeper...",True,True,Positive
4,2023-05-25 Manchester United v Chelsea,Manchester United,3837548--75,3,00:00:01.781000000+00:00,00:00:05.343000000+00:00,12.749972,38.249917,10.921876,32.765629,6.13,Goal Keeper,No Touch,"[Pass, Ball Receipt*, Shot (Goal), Goal Keeper...",True,True,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45452,2023-05-28 Leicester City v West Ham United,Leicester City,3837604--160,5,00:00:02.940400000+00:00,00:00:14.702000000+00:00,30.654161,153.270807,26.554881,132.774405,9.03,Player Off,None,"[Ball Receipt*, Injury Stoppage, Injury Stoppa...",False,False,Negative
45453,2023-05-28 Leicester City v West Ham United,Leicester City,3837604--162,1,00:00:00.496000000+00:00,00:00:00.496000000+00:00,14.328294,14.328294,12.535864,12.535864,25.27,Block,None,"[Carry, Pass (Incomplete), Ball Receipt* (Inco...",False,False,Positive
45454,2023-05-28 Leicester City v West Ham United,Leicester City,3837604--163,2,00:00:00.830500000+00:00,00:00:01.661000000+00:00,6.590527,13.181054,5.720006,11.440012,6.89,Block,None,"[Carry, Pass (Incomplete), Ball Receipt* (Inco...",False,False,Positive
45455,2023-05-28 Leicester City v West Ham United,West Ham United,3837604--167,6,00:00:02.606499999+00:00,00:00:15.639000000+00:00,22.396236,134.377414,19.321011,115.926063,7.41,Dispossessed,None,"[Pass, Ball Receipt*, Carry, Dispossessed]",False,False,Negative


The `outcome` is **positive** when:
* The possession contains a shot (on target or off target)
* The team from the first possession also perform the first event of the next possession
* A foul is won by the possession team

The `outcome` is **negative** when:
* The last event is an incomplete pass
* One of the following types are in the last four events (`0..3`) of the sequence:
  * 'Dispossessed', 'Miscontrol', 'Interception', 'Clearance', 'Error', 'Pressure']
* The first event in the next possessions is performed by the opposition